In [1]:
from src.chat import Chat
from app.models.UserInput import UserInput

In [ ]:
chat = Chat()


## Set System Prompt

In [48]:
user_message = "Suggests me some articles about banana"

chat_id = "abc"
user_input = UserInput(
    chat_id=chat_id,
    message=user_message,
    currentGraph=None
)

# Goal
1. Give "Keyword node"
2. Give "GrapLink from GraphData"

In [49]:
### FIX HERE
system_template_prompt = """You are an AI assistance for encouraging the system to initialize new "node" from below data that was fetched from QdrantDB. Node have two types which are "Paper" and "Keyword".
Keyword is a short word that related to topic, title or field of the input messages from user while "Paper" is the main content of articles in the database. Your tasks are generating "Keyword" node based on summarizing the content of user messages and "Paper" node, a node that contains more . Both of them follow this structure of node object: 

Don't forget to ensure that the values that will be assigned are related to data in the databas

This is the guideines for you:
1. Analyze the given user input to find some "Keyword" that related to the user message topic, papers, articles.
2. From the paper that user give to you, You have to find the top most important 4-5 topics and keep their id, title, type, year, abstract (if the abstract too long just keep as long as you think it's important and meaningful),  author and source in Json format.
3. After you have done the step 2, Create the nodes, this is the example structure. 
    "links" : [
        
            "source" : "2410.18541_arxiv",
            "target" : "2204.13154_arxiv"
        ,
        
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2204.13154_arxiv"
        ,
        
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2410.18541_arxiv"
        ,
    ]


Ensure that every pairs of the Graphlinks member are related and as you can see from the example, some graph link might have the same source, which store
the ID of the starter node (the node that have higher level).
Don't for get to change the id of source and target to the correct id of provided paper. The IDs in the examples isn't always the same as what users provide to you.
However, You can adjust the inhertance of the graph using your decision to make it be more appropriate.
4. Node with type "keyword" will have 3 fields [id], [type] and [title] (no label)
5. In the end, Return them in json format by Nodes and GraphLinks is in the same level and re-check if everything is correct and follows the guidelines, IDs in GraphLinks are paired with the IDs from provided papers.
6. Moreover, add one more section to the response Json, "summation". It's a section to show the message that obtain from wraping the information up. You might add the title of the articles to make it be more easier to read. and don't for get to keep the conversation
in friendly mood and always encourage the user to ask you if they have any problems or need any helps.
this is the structure for summation section:
"summation" : "Text for replying to user"

However, you might be asked to find more papers once again. So, when you receive the new message from users and realize that the users need to find more data,
or rticles, you have to do the task follows the guidelines once again. Please ensure that the additional papers that will be in the response must be never given to the user,
You might re-check it by ID and title to avoid duplication. 



If you want to give dialogue or additional text please put them in "summation" don't write anything outside of the json template


"""
### FIX HERE

chat.set_system_template_prompt(system_template_prompt)

# Set Chat Template Prompt

In [51]:
### FIX HERE
chat_template_prompt = """
{paper_data}
from the above data If the user didn't change, shift, or start to any new topics and the above data isn't related the what user's talking about,
you are able to ignore those datas.

User message : {message}

[If the converson is about finding more data, do the following task. Otherwise, ignore it] :
If there is no data that related to the wanted topic that much just tell the user politly that There is no additional data that they are finding in the data base.
"""
### FIX HERE

chat.set_chat_template_prompt(chat_template_prompt)

In [52]:
### See the result of the chat

initial_result = chat.test_initial_chat(user_input=user_input)

================================ Human Message =================================


{
    "id": "85128243165.0_scopus",
    "title": "Product development of ice cream stick from raw banana powder",
    "type": "paper",
    "year": 2022,
    "abstract": "This research was focused on using raw bananas in place of natural wooden ice cream sticks. The raw bananas were used in a powdered form and then processed to create a substitute for the wooden sticks. A research operation was carried out, experimenting with different proportions of banana powder in a randomized complete block experimental design (RCBD). The properties of interest for the ice cream sticks were hardness, fracturability, and aw of samples, as well as a sensory evaluation. The research showed that ice cream sticks which contained 50\u201380% raw banana powder with 40% sugar and 40 min of baking resulted in a product with the properties of commonly used wooden ice cream sticks, and that the product also had a good appearance

In [21]:
# chat.clear_chat(chat_id=chat_id)

In [53]:
chat.get_chat_history(chat_id=chat_id)

[HumanMessage(content='\n{\n    "id": "85128243165.0_scopus",\n    "title": "Product development of ice cream stick from raw banana powder",\n    "type": "paper",\n    "year": 2022,\n    "abstract": "This research was focused on using raw bananas in place of natural wooden ice cream sticks. The raw bananas were used in a powdered form and then processed to create a substitute for the wooden sticks. A research operation was carried out, experimenting with different proportions of banana powder in a randomized complete block experimental design (RCBD). The properties of interest for the ice cream sticks were hardness, fracturability, and aw of samples, as well as a sensory evaluation. The research showed that ice cream sticks which contained 50\\u201380% raw banana powder with 40% sugar and 40 min of baking resulted in a product with the properties of commonly used wooden ice cream sticks, and that the product also had a good appearance along with positive consumer response. In addition,

In [54]:
from typing import List
from app.models.GraphData import GraphLink, Node, GraphData
import json

In [55]:
def format_chat_json_to_graph_link(chat_json_response: str) -> List[GraphLink]:
    """
    Example of chat_json_response
    ```json
    [
        {
            "source": "some_id",
            "target": "some_id",
        }
    ]
    ```
    """

    # Select content between ```json and ```
    chat_json_response = chat_json_response.split("```json")[1].split("```")[0]
    # print(chat_json_response)
    graph_data = json.loads(chat_json_response)
    # print(graph_data)
    nodes_json = graph_data["nodes"]
    links_json = graph_data["links"]

    nodes = []
    for node in nodes_json:
        if "label" in node:
            node["title"] = node["label"]
            del node["label"]
        
        if "year" not in node:
            node["year"] = None
        
        if "abstract" not in node:
            node["abstract"] = None

        if "authors" not in node:
            node["authors"] = None

        if "source" not in node:
            node["source"] = None
            
        node = Node(**node)
        nodes.append(node)

    links = []
    for link in links_json:
        link = GraphLink(**link)
        links.append(link)

    return GraphData(nodes=nodes, links=links)

In [56]:
graph_data = format_chat_json_to_graph_link(initial_result.content)

# Set Detect Additional Data

In [57]:
detect_additional_message = "Do I need more paper?"

In [58]:
user_detect_additional_data = UserInput(
    chat_id=chat_id,
    message=detect_additional_message,
    currentGraph=graph_data
)

In [59]:
detect_additional_data_template = """
If the input message that you obtain from user is asking if they need for more paper or the current status of papers that you give to them
is enough, you have to be concern the text after this but if the user is talking about the other things that's related to additional data retrieving,
just return the response in json format:
"summation": [your text that want to give back to user]

[After this, it's the part of tasks that you have to do for the situation that user asked if they need more paper or not]
These are your informative data for making the decision if user needs more papers or not:
-  User message : {message}
- This is the current graph of information that user has:
   current status or graph : {current_graph}

These are your guidelines :

Accordning to the amount, quality and usage of paper that related to the topic of the conversation. Do the user needs the more number of papers?
If there are some papers in current graph that related to the topic that user's interted in of researching.
If yes reply "Yes" Otherwise "No" followed by the reason of the answer. Furthermore, suggest the next appropriate step to the user by
considering the overall of this conversation detaily. Be calm, friendly, natural and love to give the best service to the user.
Return the response by this json template:
"isNeed" : "yes or no",
"reason" : "the reason that you suggest user to find more papers or the reason that the papaers is enough for user"


"""
chat.set_detect_additional_data_template(detect_additional_data_template)

In [60]:
detect_additional_data_result = chat.test_detect_additional_data(user_input=user_detect_additional_data)
detect_additional_data_result

================================== Ai Message ==================================

```json
{
  "isNeed": "yes",
  "reason": "While the current papers provide a good starting point, they mainly focus on specific applications of bananas, such as making ice cream sticks, energy gels, and preservation techniques.  Expanding the search could uncover articles on banana cultivation, nutritional benefits, economic impact, or other related areas, giving you a more comprehensive understanding of the topic."
}
```
None


DetectAdditionalData(isNeed=True, reason='While the current papers provide a good starting point, they mainly focus on specific applications of bananas, such as making ice cream sticks, energy gels, and preservation techniques.  Expanding the search could uncover articles on banana cultivation, nutritional benefits, economic impact, or other related areas, giving you a more comprehensive understanding of the topic.')

In [75]:
chat.clear_latest_message(chat_id=chat_id)

In [16]:
# def convert_detect_additional_data_to_boolean(detect_additional_data_response: str) -> bool:
#     return detect_additional_data_response.lower() == "yes"

In [17]:
# convert_detect_additional_data_to_boolean(detect_additional_data_result.content)

In [61]:
continue_chat_result = chat.test_continue_chat(user_input=user_detect_additional_data)

```json
{
  "isNeed": "yes",
  "reason": "The current set of papers focuses on specific applications like ice cream sticks, energy gels, and preservation. To broaden your research, exploring additional papers could reveal insights into cultivation, nutritional value, or economic aspects of bananas."
}
```
isNeed=True reason='The current set of papers focuses on specific applications like ice cream sticks, energy gels, and preservation. To broaden your research, exploring additional papers could reveal insights into cultivation, nutritional value, or economic aspects of bananas.'
================================ Human Message =================================


{
    "id": "85128243165.0_scopus",
    "title": "Product development of ice cream stick from raw banana powder",
    "type": "paper",
    "year": 2022,
    "abstract": "This research was focused on using raw bananas in place of natural wooden ice cream sticks. The raw bananas were used in a powdered form and then processed to cr

In [47]:
chat.clear_all_message(chat_id=chat_id)